# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [42]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [43]:
weather_data_to_load = "../WeatherPy/Output/weather_data.csv"
weather_data = pd.read_csv(weather_data_to_load)
weather_data = weather_data.drop(["ID"],axis=1)
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,cape town,0,ZA,1585846120,64,-33.93,18.42,79.00,12.75
1,rikitea,6,PF,1585846231,72,-23.12,-134.97,77.18,5.46
2,yaan,33,NG,1585846231,48,7.38,8.57,90.81,4.36
3,hithadhoo,38,MV,1585846132,71,-0.60,73.08,84.65,5.37
4,seoul,1,KR,1585846180,70,37.57,126.98,44.60,4.65
...,...,...,...,...,...,...,...,...,...
577,iralaya,40,HN,1585846380,70,15.00,-83.23,86.00,4.70
578,fuerte olimpo,76,PY,1585846380,31,-21.04,-57.87,99.93,4.68
579,halifax,90,CA,1585846366,100,44.65,-63.57,48.00,24.16
580,madang,99,PG,1585846380,82,-5.22,145.80,77.72,3.94


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [44]:
gmaps.configure(api_key=g_key)


In [45]:
locations = weather_data[["Lat", "Lng"]]
humidity = weather_data.Humidity

In [46]:
humidity = humidity.astype(float)

In [47]:
fig = gmaps.figure(zoom_level=1,center=(20,0),layout={'width': '600px', 'height': '600px'})

heat_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                 dissipating=False, max_intensity=100, point_radius = 1)

fig.add_layer(heat_layer)


fig

Figure(layout=FigureLayout(height='600px', width='600px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [48]:
ideal_df = weather_data[(weather_data["Max Temp"] < 80) &
                        (weather_data["Max Temp"] > 70) &
                       (weather_data["Wind Speed"] < 10) &
                       (weather_data.Cloudiness == 0)].dropna()

ideal_df = ideal_df.dropna()
ideal_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
52,emerald,0,AU,1585846243,83,-23.53,148.17,71.60,5.82
73,hermanus,0,ZA,1585846248,65,-34.42,19.23,79.00,5.01
84,lakhimpur,0,IN,1585846251,22,27.95,80.77,73.90,8.52
142,batticaloa,0,LK,1585846265,77,7.71,81.69,77.70,5.48
295,veraval,0,IN,1585846305,67,20.90,70.37,76.24,9.91
306,umm lajj,0,SA,1585846309,34,25.02,37.27,75.54,8.97
308,bajna,0,IN,1585846309,23,27.90,77.68,75.20,8.28
373,sabha,0,LY,1585846326,14,27.04,14.43,79.23,2.48
383,puerto madryn,0,AR,1585846187,19,-42.77,-65.04,74.82,8.43
429,mormugao,0,IN,1585846340,85,15.40,73.80,77.95,3.74


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [49]:
hotel_df = ideal_df[["City","Country","Lat","Lng"]].copy()
hotel_df["Hotel Name"] = ""

In [50]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
52,emerald,AU,-23.53,148.17,
73,hermanus,ZA,-34.42,19.23,
84,lakhimpur,IN,27.95,80.77,
142,batticaloa,LK,7.71,81.69,
295,veraval,IN,20.90,70.37,
306,umm lajj,SA,25.02,37.27,
308,bajna,IN,27.90,77.68,
373,sabha,LY,27.04,14.43,
383,puerto madryn,AR,-42.77,-65.04,
429,mormugao,IN,15.40,73.80,


In [70]:
for index, row in hotel_df.iterrows():
    hotel_lat = str(row["Lat"])
    hotel_lng = str(row["Lng"])
    hotel_coordinates = hotel_lat + ", " + hotel_lng
    
    params = {
        "location": hotel_coordinates,
        "keyword": "Hotel",
        "radius": 5000,
        "type": "lodging",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params)
    
    try:
        hotel_name = (response.json()["results"][0]["name"])
        hotel_df.loc[index, "Hotel Name"] = hotel_name
    except:
        print(f"Unable to find hotel for {row.City}, {row.Country}.")

Unable to find hotel for bajna, IN.


In [71]:
hotel_df

,City,Country,Lat,Lng,Hotel Name
52,emerald,AU,-23.53,148.17,Midlander Motor Inn
73,hermanus,ZA,-34.42,19.23,Hermanus Harbour Apartments
84,lakhimpur,IN,27.95,80.77,Narayan Hotel
142,batticaloa,LK,7.71,81.69,Naval Beach Villa
295,veraval,IN,20.90,70.37,Hotel The Grand Daksh Somnath
306,umm lajj,SA,25.02,37.27,Juman Hotel
308,bajna,IN,27.90,77.68,
373,sabha,LY,27.04,14.43,فندق المواجيد
383,puerto madryn,AR,-42.77,-65.04,Hotel Península Valdés
429,mormugao,IN,15.40,73.80,Hotel Nagina


In [11]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in ideal_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map


# Display Map